# Interet du calcul vectoriel
Le but de cet exemple d'illustrer l'efficacité des tableaux
numpy par rapport aux listes python.

A partir d'un nuage de points générés aléatoirement dans [0, 1[,
on cherche à identifier celui qui se trouve le plus proche d'un point M0(x0, y0).
On crée une fonction closest où les arguments sont 
un tuple (position de M0) et une liste (ou tableau) des coordonnées des points.

In [1]:
def closest(pos0, points):
    x0, y0 = pos0
    dbest, ibest = None, None # avantage de python initialiser 
                                 # avec une valeur indéfinie
    for i, (x, y) in enumerate(points):
        # carre de la distance au point M0
        d = (x - x0) ** 2 + (y - y0) ** 2
        if dbest is None or d < dbest:
            dbest, ibest = d, i 
    return ibest

## Utilisation des listes python
Comment générer une liste de coordonnées (x,y) en python, puis appeler la fonction closest

In [2]:
import random
N=100000
random.seed(123)
positions = [(random.random(), random.random()) for _ in range(N)] 
# utilisation d'une variable muette

pos0=(0.5, 0.5)
%time i=closest(pos0, positions)
%timeit i=closest(pos0, positions) 
# donne le meilleur temps = seul qui est reproductible

Wall time: 40.6 ms
42.4 ms ± 660 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Utilisation du compilateur jit du module numba

In [10]:
# ATTENTION Version dépréciée
import random
from numba import jit

@jit
def closest(pos0, points):
       x0, y0 = pos0
       dbest, ibest = None, None 
       for i, (x, y) in enumerate(points):
           # carre de la distance au point M0
           d = (x - x0) ** 2 + (y - y0) ** 2
           if dbest is None or d < dbest:
               dbest, ibest = d, i 
       return ibest

N=100000
random.seed(123)
positions = [(random.random(), random.random()) for _ in range(N)] 
#print(positions)
pos0=(0.5, 0.5)
%time i=closest(pos0, positions)
%timeit i=closest(pos0, positions)

C:\Users\toussaij\anaconda3\lib\site-packages\numba\core\ir_utils.py:2139: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'points' of function 'closest'.

For more information visit https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "..\..\..\..\..\..\..\AppData\Local\Temp\ipykernel_47488\350467145.py", line 5:
<source missing, REPL/exec in use?>

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


Wall time: 1.24 s
1.06 s ± 13.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
import random
from numba import jit
from numba.typed import List

@jit
def closest(pos0, points):
       x0, y0 = pos0
       dbest, ibest = None, None 
       for i, (x, y) in enumerate(points):
           # carre de la distance au point M0
           d = (x - x0) ** 2 + (y - y0) ** 2
           if dbest is None or d < dbest:
               dbest, ibest = d, i 
       return ibest

N=100000
random.seed(123)
positions = List()
[positions.append((random.random(), random.random())) for _ in range(N)]

#print(positions)
pos0=(0.5, 0.5)
%time i=closest(pos0, positions)
%timeit i=closest(pos0, positions)

Wall time: 110 ms
316 µs ± 6.65 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Le premier temps explose car il intègre la phase de compilation
Le second utilise la version compilée de la fonction, d'où le gain en performance

## Utilisation d'un tableau numpy
Comment générer un tableau numpy 2D , puis appeler la fonction closest


In [15]:
import numpy as np  

# tableau numpy sans jit
def numpy_closest(pos0, points):
    x0, y0 = pos0
    x,  y  = points[:,0], points[:,1]
    d = (x - x0) ** 2 + (y - y0) ** 2    
    return d.argmin()

N=100000
np.random.seed(123)
positions = np.random.rand(N, 2)
#print(positions)
#print(type(positions))
pos0=(0.5, 0.5)
%timeit i=closest(pos0, positions)
%timeit i=numpy_closest(pos0, positions)

253 µs ± 865 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.01 ms ± 3.92 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Le gain de performance est du à la suppression de la boucle explicite

Utilisation du compilateur jit du module numba

In [18]:
import random
import numpy as np  
from numba import jit

@jit
def numpy_closest(pos0, points):
    x0, y0 = pos0
    x,  y  = points[:,0], points[:,1]
    d = (x - x0) ** 2 + (y - y0) ** 2    
    return d.argmin()

N=100000
np.random.seed(123)
positions = np.random.rand(N, 2)
#print(positions)
#print(type(positions))
pos0=(0.5, 0.5)
%timeit i=closest(pos0, positions)
%timeit i=numpy_closest(pos0, positions)

254 µs ± 1.89 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
179 µs ± 4.33 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## Utilisation d'un arbre 
A titre informel, la libraire scipy contient une recherche de plus proches voisins optimisée
à base d'arbre binaire

In [19]:
import scipy.spatial as spatial
import random

N=100000
np.random.seed(123)
positions = np.random.rand(N, 2)

kdtree = spatial.KDTree(positions)
pos0=(0.5, 0.5)
%timeit d, i = kdtree.query(pos0, 1)
d, i = kdtree.query(pos0, 1)
print(i)
print(positions[i, :])

30.7 µs ± 313 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
61502
[0.49981353 0.49615511]
